<a href="https://colab.research.google.com/github/PTson2207/Classification-WBC/blob/main/First_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import csv
import cv2
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import  Adam
from keras.utils import  np_utils
from keras.preprocessing.image import  ImageDataGenerator
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
num_classes = 5
epochs = 50
BASE_DIR = '../'
batch_size = 32

In [3]:
def get_data(folder):
    X = []
    y = []

    for img in os.listdir(folder):
        if not img.startswith('.'):
            for image_filename in os.listdir(folder + img):
                img_file = cv2.imread(folder + img + '/' + image_filename)
                if img_file is not None:
                    img_file = cv2.resize(img_file, (160,120))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(img)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [4]:
X_train, y_train = get_data(BASE_DIR + "/content/drive/MyDrive/WBC_classification/Data/TRAIN/")

In [5]:
print("X_train shape:",X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (5000, 120, 160, 3)
y_train shape: (5000,)


In [6]:
X_test, y_test = get_data(BASE_DIR + "/content/drive/MyDrive/WBC_classification/Data/TEST/")

In [7]:
print("X_test shape:",X_test.shape)
print("y_test shape:",y_test.shape)

X_test shape: (1733, 120, 160, 3)
y_test shape: (1733,)


In [ ]:
X_val, y_val = get_data(BASE_DIR + "/content/drive/MyDrive/WBC_classification/Data/VALID/")

In [ ]:
print("X_val shape:", X_val.shape)
print('y_val shape:', y_val.shape)

In [ ]:
import matplotlib.pyplot as plt

# Plot 9 images
for i, image in enumerate(X_train[:9]):
    # define subplot
    plt.subplot(330 + 1 + i)
    plt.imshow(image)
    print('image:', image.shape, 'label:', y_train[i])
# show the figure
plt.show()

In [ ]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=0.0001),
                metrics=['accuracy'])

    return model


In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_test = encoder.transform(y_test)
encoded_y_val = encoder.transform(y_val)

y_train = np_utils.to_categorical(encoded_y_train)
y_test = np_utils.to_categorical(encoded_y_test)
y_val = np_utils.to_categorical(encoded_y_val)

In [ ]:
print(y_train)

In [ ]:
print(y_train.shape)

In [ ]:
model = get_model()

# fits the model on batches
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,
    shuffle=True,
    batch_size=32,
    callbacks=[remote, early_stopping],
    validation_data=(X_val, y_val))

In [ ]:
y_pred = np.rint(model.predict(X_test))

In [ ]:
print("Test Acc score:", accuracy_score(y_test, y_pred))

In [ ]:
#y_test = np.argmax(y_test, axis=1)
#y_pred = np.argmax(y_pred, axis=1)
#print(confusion_matrix(y_test, y_pred))